In [1]:
using Symbolics
using Latexify

**Demonstration of symbolic differentiation limitations**

We make use of the famous logistic map example which is defined by the following iterative function definition: $$\mathcal{L}_{i+1}(w) = 4 \mathcal{L}_{i}(w)(1-\mathcal{L}_{i}(w)), \quad \mathcal{L}_{1}(w)=w .$$ 

In [2]:
# Define symbolic variable w and its differential D
@variables w
D = Differential(w);

In [3]:
# Define the logistic map function for different orders
L_1 = w
L_2 = 4*w*(1-w)
L_3 = 16*w*(1-w)*(1-2*w)^2
L_4 = 64*w*(1-w)*(1-2*w)^2*(1-8*w+8*w^2)^2;

Next, we study the resulting first-order derivatives.

In [4]:
expand_derivatives(D(L_1))

1

In [5]:
expand_derivatives(D(L_2))

-4w + 4(1 - w)

In [6]:
expand_derivatives(D(L_3))

16(1 - w)*((1 - 2w)^2) - 16w*((1 - 2w)^2) - 64(1 - 2w)*w*(1 - w)

In [7]:
expand_derivatives(D(L_4))

-64w*((1 - 2w)^2)*((1 - 8w + 8(w^2))^2) + 64(1 - w)*((1 - 2w)^2)*((1 - 8w + 8(w^2))^2) - 256(1 - 2w)*w*(1 - w)*((1 - 8w + 8(w^2))^2) + 128(-8 + 16w)*w*(1 - w)*((1 - 2w)^2)*(1 - 8w + 8(w^2))

**Observation**: the number of terms with the derivative increases exponentially with the order $i$ of $\mathcal{L}_{i}(w)$ (so-called *expression swell*).

Next, we are going to try to differentiate through an ODE solution obtained by a Runge-Kutta (RK) solver.

In [13]:
using DifferentialEquations
@variables x0
f = (x,p,t) -> 2x # simple ODE d/t*x(t)=2*x(t)
prob = ODEProblem(f,x0,(0.0,1.0))
sol = solve(prob,RK4(), dt=1/10,adaptive=false) # analytical solution is x(t)= x0*exp(2t)

retcode: Success
Interpolation: 3rd order Hermite
t: 11-element Vector{Float64}:
 0.0
 0.1
 0.2
 0.30000000000000004
 0.4
 0.5
 0.6
 0.7
 0.7999999999999999
 0.8999999999999999
 1.0
u: 11-element Vector{Num}:
 x0
  1.2214x0
  1.49181796x0
  1.8221064563440001x0
  2.225520825778562x0
  2.7182511366059354x0
  3.3200719382504893x0
  4.055135865379148x0
  4.952942945974091x0
  6.049524514212755x0
  7.388889241659459x0

Quite cool intermediate solution: Julia is able to output the RK4 solution of the ODE with a symbolic starting condition. And we are even able to provide a derivative of the ODE solution, e.g., at the 2nd time step.

In [15]:
Dx = Differential(x0);
sol.u[2]

1.2214x0

In [16]:
expand_derivatives(Dx(sol.u[2]))

1.2214

In this example, we have used RK4 with a fixed step size as configured via `dt=1/10,adaptive=false`. Let's see what is happening if we drop these options such that the RK4 solver can choose its own step size.

In [17]:
sol = solve(prob,RK4()) 

LoadError: TypeError: non-boolean (Num) used in boolean context

Due to the adaptive step size degree of freedom within the solver, the resulting adaptive algorithm cannot be traced into a symbolic expression as the number of RK4 iterations is unknown before run time. 

**Symbolic differentiation is therefore limited to problems which are representable by symbolic algorithms, that is, quasi-static code.**